# Kunstmatige intelligentie

Naive Bayes classificatie en genetische algoritmen

## Tekstclassificatie

![Spam](images/22/spam.jpg)

Spam of geen spam? Welk label kan je op een bericht plakken, spam of inbox?

### Welke auteur?

![The Cuckoo's Calling ](images/22/Cuckoo.png)

![Gailbraith / Rowling](images/22/galbraith_rowling.png)

Welke mogelijke auteur is de schrijver van een onbekende tekst? [The Cuckoo's Calling](https://en.wikipedia.org/wiki/The_Cuckoo%27s_Calling) van Robert Galbraith bleek geschreven door [J.K. Rowling](https://www.scientificamerican.com/article/how-a-computer-program-helped-show-jk-rowling-write-a-cuckoos-calling/)!

## Kansen!

![Dobbelstenen](images/22/dice-6.png)

Dit wordt een verhaal over kansen! De centrale vraag is, gegeven teksten die eerder zijn gezien (en waar de auteur, of het *label* van bekend is), wat kunnen we zeggen over de waarschijnlijkheid dat een auteur de schrijver is van een tekst die niet eerder is gezien?

![Bayes spam filter](images/22/bayesian_spam_filter.png)

### Modellen

Een model is een vereenvoudigde weergave van een werkelijkheid

```python
{
    "love": 50
    "spell": 8
    "thou": 42
}
```

Woordfrequentie is een maat voor de "stijl" van een auteur

Het is niet het enige dat stijl kenmerkt, zo zijn meer kenmerken te bedenken (bijvoorbeeld het gebruik van punctuatie). Het is een vereenvoudigd maar informatief genoeg model van wat een auteur van andere auteurs onderscheidt. Wat het ene model van het andere onderscheid worden vaak "features" genoemd, in dit geval de tellingen van woordgebruik.

### Modellen vergelijken?

Tekst van Robert Galbraith

```python
{   # Robert Galbraith, geen "potter" ...
    "love": 50
    "spell": 8
    "thou": 42
}
```

Tekst van J.K. Rowling

```python
{   # J.K. Rowling, geen "thou" ...
    "love": 25
    "spell": 275
    "potter": 700
}
```

Nieuwe tekst, auteur onbekend

```python
{   # Onbekend auteur...
    "love": 3
    "potter": 2
    "thou": 1
    "spam": 4
}
```

Hoe zou je de "gelijkheid" van de nieuwe tekst met de twee bekende modellen kunnen scoren?

### Naive Bayes classification

![Theorema van Bayes](images/22/bayes_theorem.png)

Let niet teveel op de vergelijking, we gaan deze vereenvoudigd gebruiken!

#### Naïviteit

Aanname: elk woord in een zin is *onafhankelijk* van de ander

> this party was fun

of

> party fun was this

maakt niet uit!

Dat de volgorde van woorden niet uitmaakt is een naïeve aanname, maar wel handig voor ons probleem.

### Stappen

Normaliseer de modellen van de bekende auteurs

```python
{   # Robert Galbraith
    "love": 0.5    # 50
    "spell": 0.08  # 8
    "thou": 0.42   # 42
}
```

```python
{   # J.K. Rowling
    "love": 0.025   # 25
    "spell": 0.275  # 275  
    "potter": 0.7   # 700
}
```

### Model *scores*

Bereken een score per model voor de onbekende tekst

```python
{   # Robert Galbraith
    "love": 0.5
    "spell": 0.08
    "thou": 0.42
}
```

```python
{   # Onbekend
    "love": 3
    "potter": 2
    "thou": 1
    "spam": 4
}
```

De kans dat deze woorden in deze aantallen samen voorkomen is voor "love" (3x) en "thou" (1x)

$$
0.5 \times 0.5 \times 0.5 \times 0.42 \times ?
$$

Wat te kiezen voor de ontbrekende woorden "potter" (2x) en "spam (4x)"?

#### Ontbrekende waarden

Gebruik 0 als waarde?

$$
0.5 \times 0.5 \times 0.5 \times 0.42 \times 0 \times 0 \times 0 \times 0 \times 0 \times 0 = 0
$$

Gebruik 1 als waarde?

$$
0.5 \times 0.5 \times 0.5 \times 0.42 \times 1 \times 1 \times 1 \times 1 \times 1 \times 1 = 0.525
$$

Een waarde 0 is in in ieder geval niet nuttig. Maar waarom is vooral dit geen goed idee om 1 te gebruiken?

#### Oplossing

Gebruik de *helft* van de *kleinste waarde* voor beide modellen

```python
{   # Robert Galbraith
    "love": 0.5
    "spell": 0.08
    "thou": 0.42
}
{   # J.K. Rowling
    "love": 0.025
    "spell": 0.275  
    "potter": 0.7
}
```

De score voor Robert Galbraith:

$$
0.5 \times 0.5 \times 0.5 \times 0.42 \times 0.012 \times 0.012 \times 0.012 \times 0.012 \times 0.012 \times 0.012 = 1.57\text{e-}13
$$

De kleinste waarde is $0.025$ ("love") en deze waarde gedeeld door twee is ongeveer gelijk aan $0.012$. Dit is niet de enige manier om met 0-waarden om te gaan, een andere veelgebruikte techniek is [Laplace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing).

De uikomst $1.57\text{e-}13$ is geschreven in een wetenschappelijke notatie en staat voor voor $1.57 \cdot 10^{-13}$. Dit zijn heel veel getallen achter de komma, we kunnen hier een andere vorm gebruiken!

### Logaritme

Gebruik het [logaritme](https://en.wikipedia.org/wiki/Binary_logarithm) van de waarden, van

$$
0.5 \times 0.5 \times 0.5 \times 0.42 \times 0.012 \times 0.012 \times 0.012 \times 0.012 \times 0.012 \times 0.012 = 1.57\text{e-}13
$$

naar 
$$
3 \times log_{2}0.5 + log_{2}0.42 + 6 \times log_{2}0.012 = -29.48
$$

### Python?

In [1]:
from math import log

score = 3 * log(0.5) + log(0.42) + 6 * log(0.012)

In [2]:
score

-29.48403388454938

```python
{   # J.K. Rowling
    "love": 0.025
    "spell": 0.275  
    "potter": 0.7
}
```

```python
{   # Onbekend
    "love": 3
    "potter": 2
    "thou": 1
    "spam": 4
}
```

De score voor J.K. Rowling

$$
3 \times log_{2}0.025 + 2 \times log_{2}0.7 + 5 \times log_{2}0.012 = -33.89
$$

### De winnaar

Het model met de beste (hoogste) score!

- Robert Galbraith: −29.48 (de (veel?) betere match)
- J.K. Rowling: −33.89

## Genetische algoritmen

![Picobot Full Circle](images/22/picobot_full_circle.png)

De cirkel is nu rond, je bent begonnen met Picobot en kan er nu mee eindigen!

### Picobot

Het idee:

1. Implementeer Picobot in Python
2. Train Python om succesvolle Picobot programma's te schrijven!

### Regels

![Picobot rules](images/22/picobot_options.png)

![Picobot full circle](images/22/picobot.gif)

```text
0 xxxx -> N 0
0 Nxxx -> W 0
0 NxWx -> S 0
0 xxWx -> S 0
0 xxWS -> E 0
0 xxxS -> E 0
0 xExS -> N 0
0 xExx -> N 0
0 NExx -> S 1
1 xxxx -> S 1
1 Nxxx -> E 1
1 NxWx -> E 1
1 xxWx -> N 1
1 xxWS -> N 1
1 xxxS -> W 1
1 xExS -> W 1
1 xExx -> S 1
1 NExx -> W 0
```

| Huidige staat | Omgeving   |   Stap   | Richting | Nieuwe staat |
|---------------|------------|:--------:|----------|--------------|
|    **`0`**    | **`NExx`** | **`->`** |  **`S`** |    **`1`**   |

```text
+++++++++++++++++++++++++    Wall:    +
+oooooPooooooooooooooooo+    Visited: o
+o              o      o+    Picobot: P
+o              o      o+
+o              o      o+
+o              o      o+
+o              o      o+
+o              o      o+
+o              o      o+
+o              o      o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+o                     o+
+ooooooooooooooooooooooo+
+++++++++++++++++++++++++
```

### Klassen

Een Picobot programma (een verzameling regels)

```python
class Program:
    ...
```

Welk type zal `self.rules` zijn?

```text
1 NExx -> W 0
```

```python
self.rules[(1, "NEXX")] = ("W", 0)
```

Een dictionary is een geschikt type waar de "huidige staat" en "omgeving" de sleutel is en *richting* en *nieuwe staat* als waarde, beide als tuples.

Een Picobot wereld (de ruimte en de positie van Picobot)

```python
class World:
    ...
```

Welk type zal `self.room` zijn?

Een list of lists voor een 2D array!

- Muur: `+`
- Bezocht: `o`
- Picobot: `P`

## De evolutie van Picobot

Een voorbeeld van een een genetisch algoritme voor het optimaliseren van

- *moeilijk te beschrijven* functies
- met behulp van *eenvoudig op te delen* oplossingen

![Genetic algorithm](images/22/genetic_algorithm.png)

### Evolutie

Begin met een populatie van ~200 *random* Picobot programma's

![Evo 1a](images/22/evo_1a.png)

### Fitness

Bepaal per Picobot programma de dekking als waarde tussen 0 en 1

![Evo 1b](images/22/evo_1b.png)

### Combineren

Paar (*crossover*) en muteer de meest fitte 10-20% programma's en begin weer met ~200 random Picobot programma's

![Evo 1c](images/22/evo_1c.png)

### Herhalen

Herhaal dit "survival of the fittest" proces voor meerdere generaties tot een meest optimale fit is gevonden!